In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import seaborn as sn
import teotil2 as teo
import useful_rid_code as rid

sn.set_context("notebook")

# RID

## Estimating loads in unmonitored regions (parameterised)

This notebook is "parameterised" for use with Papermill. The cell below has the tag `parameters`, which means the entire notebook can be called from `01_recalculate_ospar_1990-2016_main.ipynb`.

The [TEOTIL2 model](https://nivanorge.github.io/teotil2/) is used to estimate loads in unmonitored areas. We know the regine ID for each of the 155 stations where water chemistry is measured, and we also know which OSPAR region each monitoring site drains to. We want to use observed data to estimate loads upstream of each monitoring point, and modelled data elsewhere.

In [2]:
# This cell is tagged 'parameters' for use with Papermill
# https://papermill.readthedocs.io/en/latest/index.html
year = 1990
user = ""
pw = ""

In [3]:
# Parameters
user = "jes"
pw = "BeakabusNov21.."
year = 2016


## 1. Generate model input file

In [4]:
# Parameters of interest
par_list = ["Tot-N", "Tot-P"]

# Path to TEOTIL2 "core" input data
teo_fold = r"../../../teotil2/data/core_input_data"

# Ouput path for model file
ann_input_csv = f"../../../teotil2/data/norway_annual_input_data/input_data_{year}.csv"

In [5]:
engine = rid.connect_to_nivabase(user=user, pw=pw)
df = teo.io.make_input_file(
    year, engine, teo_fold, ann_input_csv, mode="nutrients", par_list=par_list
)

Connection successful.


/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


## 2. Run model

In [6]:
%%time

# Run model
g = teo.model.run_model(ann_input_csv)

/opt/conda/lib/python3.9/site-packages/teotil2/model.py:67: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  df[f"trans_{par}"].between(0, 1, inclusive=True).all()


CPU times: user 6.32 s, sys: 50.4 ms, total: 6.38 s
Wall time: 6.43 s


## 3. Save results

In [7]:
# Save results as csv
res_csv = f"../../../teotil2/data/norway_annual_output_data/teotil2_results_{year}.csv"
df = teo.model.model_to_dataframe(g, out_path=res_csv)

df.head()

,regine,regine_ned,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,...,local_ren_tot-n_tonnes,local_ren_tot-p_tonnes,local_runoff_mm/yr,local_spr_tot-n_tonnes,local_spr_tot-p_tonnes,local_trans_tot-n,local_trans_tot-p,local_urban_tot-n_tonnes,local_urban_tot-p_tonnes,local_vol_lake_m3
0,001.1A2B,001.1A2A,2.797730,0.177308,0.033277,0.002690,0.327863,0.022596,12.865020,0.512309,...,0.0,0.0,263.309382,0.294586,0.019906,1.00,1.00,0.02205,0.00315,1.788754e+08
1,001.1A4D,001.1A4C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.238356,0.008723,...,0.0,0.0,307.194279,0.000000,0.000000,0.81,0.26,0.00000,0.00000,8.019497e+06
2,001.1M,001.1L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.095496,0.032064,...,0.0,0.0,307.194279,0.000000,0.000000,0.86,0.31,0.00000,0.00000,3.866908e+07
3,001.221Z,001.2210,0.112875,0.007154,0.001343,0.000109,0.013228,0.000912,0.364509,0.010769,...,0.0,0.0,263.309382,0.011885,0.000803,1.00,1.00,0.00000,0.00000,0.000000e+00
4,001.222Z,001.2220,0.930145,0.058948,0.011063,0.000894,0.109003,0.007512,1.998284,0.078689,...,0.0,0.0,263.309382,0.097939,0.006618,1.00,1.00,0.00000,0.00000,0.000000e+00


In [8]:
# Save version with main catchments only
main_list = ["%03d." % i for i in range(1, 316)]
df2 = df.query("regine in @main_list")
df2.sort_values("regine", inplace=True)

# Save
main_csv = f"../../../Results/Unmon_loads/teotil2_results_{year}_main_catchs.csv"
df2.to_csv(main_csv, index=False, encoding="utf-8")

/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


## 4. Explore results

### 4.1. Total N and P

####  4.1.1. Identify areas with monitoring data

Where observations are available, we want to use them in preference to the model output. This means identifying all the catchments with observed data and substracting the model results for these locations. This is more complicated than it appears, because a small number of observed catchments are upstream of others, so subtracting all the loads for the 155 monitored catchments involves "double accounting", which we want to avoid. The first step is therefore to identify the downstream-most nodes for the monitored areas i.e. for the cases where one catchment is upstream of another, we just want the downstream node.

In [9]:
# Read station data
in_xlsx = r"../../../Data/RID_Sites_List_2017-2020.xlsx"
stn_df = pd.read_excel(in_xlsx, sheet_name="RID_All")
stn_df = stn_df.query("station_id != 38005")  # Ignore TROEMÅL2

# Get just cols of interest and drop duplicates
# (some sites are in the same regine)
stn_df = stn_df[["ospar_region", "nve_vassdrag_nr"]].drop_duplicates()

# Get catch IDs with obs data
obs_nds = set(stn_df["nve_vassdrag_nr"].values)

# Build network from input file
g, nd_list = teo.calib.build_calib_network(ann_input_csv, obs_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df["nve_vassdrag_nr"].isin(ds_nds)]

#### 4.1.2. Sum model results for monitored locations

In [10]:
# Read model output
teo_df = pd.read_csv(res_csv)

# Join accumulated outputs to stns of interest
mon_df = pd.merge(
    stn_df, teo_df, how="left", left_on="nve_vassdrag_nr", right_on="regine"
)

# Groupby OSPAR region
mon_df = mon_df.groupby("ospar_region").sum()

# Get just accum cols
cols = [i for i in mon_df.columns if i.split("_")[0] == "accum"]
mon_df = mon_df[cols]

mon_df.head()

,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2,accum_urban_tot-n_tonnes,accum_urban_tot-p_tonnes
ospar_region,,,,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,139.951511,5.259556,2.539554,0.218376,77.940831,5.693310,3437.382905,51.234899,139.951511,5.259556,...,3219.490562,40.282033,740.402957,24.647655,0.757696,50.753621,4.717238,63555.61,0.000000,0.000000
NORTH SEA,2570.998021,67.336920,34.862152,2.176470,422.583107,40.109738,11384.514637,178.281933,2576.970591,67.880396,...,8384.960940,70.291800,1197.961378,232.678647,24.009273,129.438464,10.325821,23353.19,5.972570,0.543476
NORWEGIAN SEA2,2683.049873,72.430596,40.763819,3.061545,577.559392,52.806835,10555.706132,228.121741,2699.516851,74.559747,...,7278.629890,100.755160,1341.437751,297.559847,18.818656,193.193896,20.464933,45896.63,16.466978,2.129151
SKAGERAK,10972.326142,226.858874,102.079627,5.546824,3852.921376,111.010225,27856.550732,472.779256,11134.261381,244.336637,...,12869.367975,117.432394,1563.684758,2696.074742,25.094948,872.681816,52.029478,93945.27,161.935239,17.477763


This table gives the **modelled** inputs to each OSPAR region from catchments for which we have observed data. We want to subtract these values from the overall modelled inputs to each region and substitute the observed data instead.

The trickiest part of this is that the OSPAR regions in the TEOTIL catchment network (files named `regine_{year}.csv`) don't exactly match the relevant OSPAR definitions for this analysis. This is because the "OSPAR boundaries" in the model include catchments draining to Sweden (as part of TEOTIL2 Metals - see [here](https://nivanorge.github.io/teotil2/pages/07_1000_lakes.html)), so instead of using them directly we need to aggregate based on vassdragsnummers.

#### 4.1.3. Group model output according to "new" OSPAR regions

In [11]:
# Define "new" OSPAR regions (ranges are inclusive)
os_dict = {
    "SKAGERAK": (1, 23),
    "NORTH SEA": (24, 90),
    "NORWEGIAN SEA2": (91, 170),
    "LOFOTEN-BARENTS SEA": (171, 247),
}

# Container for results
df_list = []

# Loop over model output
for reg in os_dict.keys():
    min_id, max_id = os_dict[reg]

    regs = ["%03d." % i for i in range(min_id, max_id + 1)]

    # Get data for this region
    df2 = teo_df[teo_df["regine"].isin(regs)]

    # Get just accum cols
    cols = [i for i in df2.columns if i.split("_")[0] == "accum"]
    df2 = df2[cols]

    # Add region
    df2["ospar_region"] = reg

    # Add to output
    df_list.append(df2)

# Build df
os_df = pd.concat(df_list, axis=0)

# Aggregate
os_df = os_df.groupby("ospar_region").sum()

os_df.head()

,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2,accum_urban_tot-n_tonnes,accum_urban_tot-p_tonnes
ospar_region,,,,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,585.376148,27.603191,10.565114,0.885819,15539.558643,2624.535584,23224.852465,2761.427407,595.348458,29.007032,...,7089.945363,107.884791,1773.305858,975.416656,141.694898,329.527630,40.781418,138090.89,9.972310,1.403841
NORTH SEA,6188.087700,186.930657,80.323926,5.829535,24165.226167,3944.860872,47778.657576,4300.956581,6227.905182,191.999327,...,17385.526226,164.096381,2564.298056,3389.982318,393.908106,696.254634,81.460852,59314.38,39.817482,5.068670
NORWEGIAN SEA2,7335.129336,232.403497,101.708615,8.082202,27426.509913,4531.325089,48722.837340,4973.150948,7367.288126,236.643940,...,13929.039301,205.181920,2738.570912,2614.889278,344.770650,716.427779,87.515367,113934.05,32.158791,4.240442
SKAGERAK,12595.471146,287.465832,112.985724,6.352341,10610.537347,276.971139,37436.750691,726.528340,12869.205614,320.197798,...,13957.007729,129.359403,1659.006118,8378.982285,127.767500,1036.363378,69.405388,102574.69,273.734469,32.731966


We can now calculate the unmonitored component by simply subtracting the values modelled upstream of monitoring stations from the overall modelled inputs to each OSPAR region.

#### 4.1.4. Estimate loads in unmonitored areas

In [12]:
# Calc unmonitored loads
unmon_df = os_df - mon_df

# Write output
out_csv = f"../../../Results/Unmon_loads/teotil2_raw_unmonitored_loads_{year}.csv"
unmon_df.to_csv(out_csv, encoding="utf-8", index_label="ospar_region")

unmon_df.round(0).astype(int).T

ospar_region,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
accum_agri_diff_tot-n_tonnes,445,3617,4652,1623
accum_agri_diff_tot-p_tonnes,22,120,160,61
accum_agri_pt_tot-n_tonnes,8,45,61,11
accum_agri_pt_tot-p_tonnes,1,4,5,1
accum_all_point_tot-n_tonnes,15462,23743,26849,6758
accum_all_point_tot-p_tonnes,2619,3905,4479,166
accum_all_sources_tot-n_tonnes,19787,36394,38167,9580
accum_all_sources_tot-p_tonnes,2710,4123,4745,254
accum_anth_diff_tot-n_tonnes,455,3651,4668,1735
accum_anth_diff_tot-p_tonnes,24,124,162,76


#### 4.1.5. Aggregate values to required quantities

In [13]:
# Aggregate to match report
unmon_df["flow"] = unmon_df["accum_q_m3/s"] * 60 * 60 * 24 / 1000.0  # 1000s m3/day

unmon_df["sew_n"] = (
    unmon_df["accum_ren_tot-n_tonnes"] + unmon_df["accum_spr_tot-n_tonnes"]
)
unmon_df["sew_p"] = (
    unmon_df["accum_ren_tot-p_tonnes"] + unmon_df["accum_spr_tot-p_tonnes"]
)

unmon_df["ind_n"] = unmon_df["accum_ind_tot-n_tonnes"]
unmon_df["ind_p"] = unmon_df["accum_ind_tot-p_tonnes"]

unmon_df["fish_n"] = unmon_df["accum_aqu_tot-n_tonnes"]
unmon_df["fish_p"] = unmon_df["accum_aqu_tot-p_tonnes"]

unmon_df["diff_n"] = (
    unmon_df["accum_anth_diff_tot-n_tonnes"] + unmon_df["accum_nat_diff_tot-n_tonnes"]
)
unmon_df["diff_p"] = (
    unmon_df["accum_anth_diff_tot-p_tonnes"] + unmon_df["accum_nat_diff_tot-p_tonnes"]
)

new_df = unmon_df[
    ["flow", "sew_n", "sew_p", "ind_n", "ind_p", "fish_n", "fish_p", "diff_n", "diff_p"]
]

# Total for Norway
new_df.loc["NORWAY"] = new_df.sum(axis=0)

# Reorder rows
new_df = new_df.reindex(
    ["NORWAY", "LOFOTEN-BARENTS SEA", "NORTH SEA", "NORWEGIAN SEA2", "SKAGERAK"]
)

new_df.round().astype(int)

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p
ospar_region,,,,,,,,,
NORWAY,336242,13641,1131,2354,246,56691,9781,31118,664
LOFOTEN-BARENTS SEA,89243,1230,177,83,6,14141,2435,4326,91
NORTH SEA,118051,3724,441,412,90,19561,3370,12651,218
NORWEGIAN SEA2,120712,2841,393,984,109,22964,3971,11318,267
SKAGERAK,8236,5847,120,875,41,25,4,2823,88


## 5. Other N and P species

Tore's procedure `RESA2.FIXTEOTILPN` defines simple correction factors for estimating PO4, NO3 and NH4 from total P and N. The table below lists the factors used.

|   Source    | Phosphate | Nitrate | Ammonium |
|:-----------:|:---------:|:-------:|:--------:|
|    Sewage   |     0.600 |   0.050 |    0.750 |
|   Industry  |     0.600 |   0.050 |    0.750 |
| Aquaculture |     0.690 |   0.110 |    0.800 |
|   Diffuse   |     0.246 |   0.625 |    0.055 |


In [14]:
# Dict of conversion factors
con_dict = {
    ("sew", "po4"): ("p", 0.6),
    ("ind", "po4"): ("p", 0.6),
    ("fish", "po4"): ("p", 0.69),
    ("diff", "po4"): ("p", 0.246),
    ("sew", "no3"): ("n", 0.05),
    ("ind", "no3"): ("n", 0.05),
    ("fish", "no3"): ("n", 0.11),
    ("diff", "no3"): ("n", 0.625),
    ("sew", "nh4"): ("n", 0.75),
    ("ind", "nh4"): ("n", 0.75),
    ("fish", "nh4"): ("n", 0.8),
    ("diff", "nh4"): ("n", 0.055),
}

# Apply factors
for src in ["sew", "ind", "fish", "diff"]:
    for spc in ["po4", "no3", "nh4"]:
        el, fac = con_dict[(src, spc)]
        new_df[src + "_" + spc] = fac * new_df[src + "_" + el]

new_df.round().astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,336242,89243,118051,120712,8236
sew_n,13641,1230,3724,2841,5847
sew_p,1131,177,441,393,120
ind_n,2354,83,412,984,875
ind_p,246,6,90,109,41
fish_n,56691,14141,19561,22964,25
fish_p,9781,2435,3370,3971,4
diff_n,31118,4326,12651,11318,2823
diff_p,664,91,218,267,88
sew_po4,679,106,265,236,72


## 6. Other quantities

The model currently only considers N and P, but the project focuses on a wider range of parameters. For now, we simply assume that all measured inputs (`renseanlegg`, `industri` and `akvakultur`) for regines outside of catchments with measured data make it to the sea.

We only want data for catchments that are not monitored i.e. for regine IDs **not** in the graph created above.

In [15]:
engine = rid.connect_to_nivabase(user=user, pw=pw)

# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = (
    "SELECT SUBSTR(a.regine, 1, 3) AS vassdrag, "
    "  a.type, "
    "  b.name, "
    "  b.unit, "
    "  SUM(c.value * d.factor) as value "
    "FROM RESA2.RID_PUNKTKILDER a, "
    "RESA2.RID_PUNKTKILDER_OUTPAR_DEF b, "
    "RESA2.RID_PUNKTKILDER_INPAR_VALUES c, "
    "RESA2.RID_PUNKTKILDER_INP_OUTP d "
    "WHERE a.anlegg_nr = c.anlegg_nr "
    "AND (1, a.regine) NOT IN %s "
    "AND d.in_pid = c.inp_par_id "
    "AND d.out_pid = b.out_pid "
    "AND c.year = %s "
    "GROUP BY SUBSTR(a.regine, 1, 3), a.type, b.name, b.unit "
    "ORDER BY SUBSTR(a.regine, 1, 3), a.type" % (tuple(nd_list_hack), year)
)

df = pd.read_sql(sql, engine)

# Tidy
df["par"] = df["type"] + "_" + df["name"] + "_" + df["unit"]
del df["name"], df["unit"], df["type"]

# Pivot
df = df.pivot(index="vassdrag", columns="par", values="value")
df.reset_index(inplace=True)

Connection successful.


In [16]:
def f(x):
    try:
        a = int(x)
        return a
    except:
        return -999


# Convert vassdrag to numbers
df["vass"] = df["vassdrag"].apply(f)

# Get just the main catchments
df = df.query("vass != -999")

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PCB_tonn,...,RENSEANLEGG_Hg_tonn,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass
0,001,0.048950,0.013790,0.007210,0.113090,2.000000e-05,NaN,NaN,0.052550,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,139.92692,14.96600,NaN,1
1,002,0.016039,0.005805,0.798983,4.320172,4.219900e-03,NaN,NaN,0.460639,NaN,...,0.002027,0.234566,0.004062,0.000152,0.022118,0.013,723.88409,12.33553,1.032589,2
2,003,0.000027,0.000014,0.000120,0.001874,2.000000e-07,NaN,NaN,0.000321,NaN,...,0.000027,0.021600,0.000000,0.000000,0.001700,NaN,289.30000,2.79800,0.104800,3
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,153.11400,1.16000,NaN,4
4,005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,31.27113,0.18367,NaN,5


In [17]:
def f2(x):
    if x in range(1, 24):
        return "SKAGERAK"
    elif x in range(24, 91):
        return "NORTH SEA"
    elif x in range(91, 171):
        return "NORWEGIAN SEA2"
    elif x in range(171, 248):
        return "LOFOTEN-BARENTS SEA"
    else:
        return np.nan


# Assign main catchments to OSPAR regions
df["osp_reg"] = df["vass"].apply(f2)

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PCB_tonn,...,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass,osp_reg
0,001,0.048950,0.013790,0.007210,0.113090,2.000000e-05,NaN,NaN,0.052550,NaN,...,NaN,NaN,NaN,NaN,NaN,139.92692,14.96600,NaN,1,SKAGERAK
1,002,0.016039,0.005805,0.798983,4.320172,4.219900e-03,NaN,NaN,0.460639,NaN,...,0.234566,0.004062,0.000152,0.022118,0.013,723.88409,12.33553,1.032589,2,SKAGERAK
2,003,0.000027,0.000014,0.000120,0.001874,2.000000e-07,NaN,NaN,0.000321,NaN,...,0.021600,0.000000,0.000000,0.001700,NaN,289.30000,2.79800,0.104800,3,SKAGERAK
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,153.11400,1.16000,NaN,4,SKAGERAK
4,005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,31.27113,0.18367,NaN,5,SKAGERAK


In [18]:
# Group by OSPAR region
df.fillna(0, inplace=True)
df = df.groupby("osp_reg").sum()
if 0 in df.index:
    df.drop(0, inplace=True)

# Total for Norway
df.loc["NORWAY"] = df.sum(axis=0)

# Join to model results
df = new_df.join(df)

# Get cols of interest
umod_cols = ["S.P.M.", "TOC", "As", "Pb", "Cd", "Cu", "Zn", "Ni", "Cr", "Hg"]
umod_cols = [
    "%s_%s_tonn" % (i, j) for i in ["INDUSTRI", "RENSEANLEGG"] for j in umod_cols
]
cols = list(new_df.columns) + umod_cols
cols.remove("RENSEANLEGG_TOC_tonn")
cols = [i for i in cols if i in df.columns]
df = df[cols]

df.round(0).astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,336242,89243,118051,120712,8236
sew_n,13641,1230,3724,2841,5847
sew_p,1131,177,441,393,120
ind_n,2354,83,412,984,875
ind_p,246,6,90,109,41
fish_n,56691,14141,19561,22964,25
fish_p,9781,2435,3370,3971,4
diff_n,31118,4326,12651,11318,2823
diff_p,664,91,218,267,88
sew_po4,679,106,265,236,72


## 7. Fish farm copper

Finally, we need to add in the Cu totals from fish farms. The method is similar to that used above, but simpler because we're only interested in one parameter.

In [19]:
engine = rid.connect_to_nivabase(user=user, pw=pw)

# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = (
    "SELECT SUBSTR(regine, 1, 3) AS vassdrag, "
    "  SUM(value) AS value FROM ( "
    "    SELECT b.regine, "
    "           c.name, "
    "           (a.value*d.factor) AS value "
    "    FROM resa2.rid_kilder_aqkult_values a, "
    "    resa2.rid_kilder_aquakultur b, "
    "    resa2.rid_punktkilder_outpar_def c, "
    "    resa2.rid_punktkilder_inp_outp d "
    "    WHERE a.anlegg_nr = b.nr "
    "    AND (1, b.regine) NOT IN %s "
    "    AND a.inp_par_id = d.in_pid "
    "    AND c.out_pid = d.out_pid "
    "    AND name = 'Cu' "
    "    AND ar = %s) "
    "GROUP BY SUBSTR(regine, 1, 3)" % (tuple(nd_list_hack), year)
)

aq_df = pd.read_sql(sql, engine)

if len(aq_df) > 0:
    # Get vassdrag
    aq_df["vass"] = aq_df["vassdrag"].apply(f)
    aq_df = aq_df.query("vass != -999")

    # Calc OSPAR region and group
    aq_df["osp_reg"] = aq_df["vass"].apply(f2)
    aq_df.fillna(0, inplace=True)
    aq_df = aq_df.groupby("osp_reg").sum()
    del aq_df["vass"]

    # Total for Norway
    aq_df.loc["NORWAY"] = aq_df.sum(axis=0)

    # Rename
    aq_df.columns = [
        "AQUAKULTUR_Cu_tonn",
    ]

    # Join model results
    df = df.join(aq_df)

    df.round(0).astype(int).T

Connection successful.


In [20]:
# Write output
out_csv = f"../../../Results/Unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
df.to_csv(out_csv)

This data can then be used to create Table 3 in the report - see [this notebook](https://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/summary_table_2017.ipynb) for details.